SVM's are great at binary classification, and have an advantage over neural networks when it comes to binary classification problems. In straight forward binary classification problems SVM will outperform even a well trained deep learning model 

In [1]:
#imports 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import numpy as np  
import tensorflow as tf
from path import Path

In [2]:
#load data
file_path = Path("../Resources/bank_telemarketing.csv")
tele_df = pd.read_csv(file_path)
tele_df.head()

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


In [3]:
#encode categorical variables using OneHotEncoder

#generate categorical variable list
tele_cat = tele_df.dtypes[tele_df.dtypes=="object"].index.tolist()

#check number of unique values
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

In [4]:
#no variables require bucketing 

#create onehotencoder instance
enc = OneHotEncoder(sparse=False)

#fit and transform 
encode_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

#add the encoded variable names to the dataframe 
encode_df.columns = enc.get_feature_names(tele_cat)
encode_df.head()

,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [5]:
#merge encoded features and drop originals
tele_df = tele_df.merge(encode_df, left_index=True, right_index=True)
tele_df = tele_df.drop(tele_cat,1)
tele_df.head()

,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [6]:
#remove loan status target from feature data
y = tele_df.Subscribed_yes.values
X = tele_df.drop(columns=["Subscribed_no", "Subscribed_yes"]).values

#split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=y)

#create standardscaler instance
scaler = StandardScaler()

#fit the scaler
X_scaler = scaler.fit(X_train)

#scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
#create svm model
svm = SVC(kernel='linear')

In [8]:
#train the svm model
svm.fit(X_train, y_train)

SVC(kernel=&#39;linear&#39;)

In [9]:
#evaluate the svm model
y_pred = svm.predict(X_test_scaled)
print(f"SVM model accuracy: {accuracy_score(y_test, y_pred):.3f}")

SVM model accuracy: 0.873


In [11]:
#define deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

#add first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

#add second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#compile the model and customize metrics 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
#train the deep nn
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

#evaluate the model 
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 22857 samples
Epoch 1/50
22857/22857 [==============================] - 5s 218us/sample - loss: 0.4054 - accuracy: 0.8679
Epoch 2/50
22857/22857 [==============================] - 4s 162us/sample - loss: 0.3726 - accuracy: 0.8735
Epoch 3/50
22857/22857 [==============================] - 3s 136us/sample - loss: 0.3703 - accuracy: 0.8735
Epoch 4/50
22857/22857 [==============================] - 3s 133us/sample - loss: 0.3691 - accuracy: 0.8735
Epoch 5/50
22857/22857 [==============================] - 3s 120us/sample - loss: 0.3683 - accuracy: 0.8735
Epoch 6/50
22857/22857 [==============================] - 3s 120us/sample - loss: 0.3676 - accuracy: 0.8735
Epoch 7/50
22857/22857 [==============================] - 3s 126us/sample - loss: 0.3671 - accuracy: 0.8735
Epoch 8/50
22857/22857 [==============================] - 3s 115us/sample - loss: 0.3667 - accuracy: 0.8735
Epoch 9/50
22857/22857 [==============================] - 3s 115us/sample - loss: 0.3662 - accuracy: 0.8735
Epoch

both models achieved an accuracy score of around 87%. Both models also take a similar amount of time to train on the input data. the only real noticeable difference is the amount of code needed to implement. The SVM requires much less code than the deep learning model. For strictly binary classification problems the SVM (support vector machine) is the better choice. 